In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import pickle
from collections import defaultdict
import random

# Load the saved model and scaler
print("Loading model...")
try:
    model = tf.keras.models.load_model(r"C:\Users\jonla\NBA_Playoffs_Series_Predictor\Models\NBA_Playoff_NN.h5")
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    print("Model loaded and compiled")
except Exception as e:
    print(f"Error loading model: {e}")
    exit(1)

print("Loading scaler...")
try:
    with open(r"C:\Users\jonla\NBA_Playoffs_Series_Predictor\Models\scaler.pkl", 'rb') as f:
        scaler = pickle.load(f)
    print(r"C:\Users\jonla\NBA_Playoffs_Series_Predictor\Models\Scaler loaded")
except Exception as e:
    print(f"Error loading scaler: {e}")
    exit(1)

# Load playoff stats
print("Loading playoff stats...")
try:
    playoff_stats_2025 = pd.read_csv(r"C:\Users\jonla\NBA_Playoffs_Series_Predictor\Data\Advanced_Stats_25.csv")
    required_columns = ['Team', 'W', 'PIE', 'eFG%', 'OREB%', 'DREB%', 'TS%', 'OffRtg', 'DefRtg']
    if not all(col in playoff_stats_2025.columns for col in required_columns):
        raise ValueError("nba_advanced_stats_2025.csv missing required columns")
    for col in ['PIE', 'eFG%', 'OREB%', 'DREB%', 'TS%']:
        if playoff_stats_2025[col].max() > 1.0:
            print(f"Normalizing {col} (dividing by 100)")
            playoff_stats_2025[col] /= 100.0
    print("Playoff stats loaded")
except Exception as e:
    print(f"Error loading playoff stats: {e}")
    exit(1)

# Function to prepare input features for prediction
def prepare_input(team1, team2, stats_df):
    try:
        t1_stats = stats_df[stats_df['Team'] == team1].iloc[0]
        t2_stats = stats_df[stats_df['Team'] == team2].iloc[0]
        
        features = {
            'T1-T2_W': t1_stats['W'] - t2_stats['W'],
            'T1-T2_PIE': t1_stats['PIE'] - t2_stats['PIE'],
            'T1-T2_eFG%': t1_stats['eFG%'] - t2_stats['eFG%'],
            'T1-T2_OREB%': t1_stats['OREB%'] - t2_stats['OREB%'],
            'T1-T2_DREB%': t1_stats['DREB%'] - t2_stats['DREB%'],
            'T1-T2_TS%': t1_stats['TS%'] - t2_stats['TS%'],
            'T1vT2_Off-Def': t1_stats['OffRtg'] - t2_stats['DefRtg'],
            'T2vT1_Off-Def': t2_stats['OffRtg'] - t1_stats['DefRtg']
        }
        
        inputs = ['T1-T2_W', 'T1-T2_PIE', 'T1-T2_eFG%', 'T1-T2_OREB%', 'T1-T2_DREB%', 
                  'T1-T2_TS%', 'T1vT2_Off-Def', 'T2vT1_Off-Def']
        X = np.array([[features[feat] for feat in inputs]])
        
        X_scaled = scaler.transform(X)
        X_scaled = np.clip(X_scaled, -5.0, 5.0)
        
        return X_scaled
    except Exception as e:
        print(f"Error preparing input for {team1} vs {team2}: {e}")
        raise

# Function to predict series win probability with symmetry
def predict_series_winner(team1, team2):
    try:
        X_scaled_t1_t2 = prepare_input(team1, team2, playoff_stats_2025)
        prob_t2_wins = model.predict(X_scaled_t1_t2, verbose=0)[0][0]
        
        X_scaled_t2_t1 = prepare_input(team2, team1, playoff_stats_2025)
        prob_t1_wins = model.predict(X_scaled_t2_t1, verbose=0)[0][0]
        
        prob_team2_wins = (prob_t2_wins + (1 - prob_t1_wins)) / 2
        return prob_team2_wins
    except Exception as e:
        print(f"Error making prediction for {team1} vs {team2}: {e}")
        return None

# Simulate a single series
def simulate_series(team1, team2):
    prob_t2_wins = predict_series_winner(team1, team2)
    if prob_t2_wins is None:
        return None
    return team2 if random.random() < prob_t2_wins else team1

# Simulate one playoff run with fixed bracket
def simulate_playoffs(east_teams, west_teams):
    # First Round
    east_first_round = [
        (east_teams[0], east_teams[7]),  # 1 vs 8
        (east_teams[3], east_teams[4]),  # 4 vs 5
        (east_teams[2], east_teams[5]),  # 3 vs 6
        (east_teams[1], east_teams[6])   # 2 vs 7
    ]
    west_first_round = [
        (west_teams[0], west_teams[7]),  # 1 vs 8
        (west_teams[3], west_teams[4]),  # 4 vs 5
        (west_teams[2], west_teams[5]),  # 3 vs 6
        (west_teams[1], west_teams[6])   # 2 vs 7
    ]
    
    east_semifinals = []
    west_semifinals = []
    
    # Simulate First Round
    for t1, t2 in east_first_round:
        winner = simulate_series(t1, t2)
        if winner is None:
            return None
        east_semifinals.append(winner)
    
    for t1, t2 in west_first_round:
        winner = simulate_series(t1, t2)
        if winner is None:
            return None
        west_semifinals.append(winner)
    
    # Conference Semifinals (fixed bracket: 1-8 vs 4-5, 2-7 vs 3-6)
    east_conf_semi = [
        (east_semifinals[0], east_semifinals[1]),  # Winner of 1 vs 8 vs Winner of 4 vs 5
        (east_semifinals[3], east_semifinals[2])   # Winner of 2 vs 7 vs Winner of 3 vs 6
    ]
    west_conf_semi = [
        (west_semifinals[0], west_semifinals[1]),
        (west_semifinals[3], west_semifinals[2])
    ]
    
    east_finals = []
    west_finals = []
    
    # Simulate Conference Semifinals
    for t1, t2 in east_conf_semi:
        winner = simulate_series(t1, t2)
        if winner is None:
            return None
        east_finals.append(winner)
    
    for t1, t2 in west_conf_semi:
        winner = simulate_series(t1, t2)
        if winner is None:
            return None
        west_finals.append(winner)
    
    # Conference Finals
    east_champ = simulate_series(east_finals[0], east_finals[1])
    west_champ = simulate_series(west_finals[0], west_finals[1])
    
    if east_champ is None or west_champ is None:
        return None
    
    # NBA Finals
    champion = simulate_series(east_champ, west_champ)
    if champion is None:
        return None
    
    return {
        'east_first_round': east_semifinals,
        'west_first_round': west_semifinals,
        'east_conf_semi': east_finals,
        'west_conf_semi': west_finals,
        'east_champ': east_champ,
        'west_champ': west_champ,
        'champion': champion
    }

# Run Monte Carlo simulation
def run_monte_carlo(east_teams, west_teams, n_simulations=10000):
    results = {
        'first_round': defaultdict(int),
        'conf_semi': defaultdict(int),
        'conf_finals': defaultdict(int),
        'champion': defaultdict(int)
    }
    
    successful_sims = 0
    
    for i in range(n_simulations):
        sim = simulate_playoffs(east_teams, west_teams)
        if sim is None:
            continue
        
        successful_sims += 1
        
        # Record First Round winners
        for team in sim['east_first_round'] + sim['west_first_round']:
            results['first_round'][team] += 1
        
        # Record Conference Semifinals winners
        for team in sim['east_conf_semi'] + sim['west_conf_semi']:
            results['conf_semi'][team] += 1
        
        # Record Conference Finals winners
        for team in [sim['east_champ'], sim['west_champ']]:
            results['conf_finals'][team] += 1
        
        # Record Champion
        results['champion'][sim['champion']] += 1
    
    if successful_sims == 0:
        print("No successful simulations")
        return None
    
    # Normalize probabilities as series win probabilities
    for team in results['first_round']:
        results['first_round'][team] = (results['first_round'][team] / successful_sims) * 100  # 8 series, 100% each
    for team in results['conf_semi']:
        results['conf_semi'][team] = (results['conf_semi'][team] / successful_sims) * 100   # 4 series, 100% each
    for team in results['conf_finals']:
        results['conf_finals'][team] = (results['conf_finals'][team] / successful_sims) * 100  # 2 series, 100% each
    for team in results['champion']:
        results['champion'][team] = (results['champion'][team] / successful_sims) * 100      # 1 series, 100%
    
    # Ensure First Round sums to 800% by setting opponent probabilities
    first_round_probs = {}
    for team in results['first_round']:
        first_round_probs[team] = results['first_round'][team]
    
    # Define First Round matchups
    west_matchups = [
        (west_teams[0], west_teams[7]),  # OKC vs Memphis
        (west_teams[3], west_teams[4]),  # Denver vs LA Clippers
        (west_teams[2], west_teams[5]),  # Lakers vs Timberwolves
        (west_teams[1], west_teams[6])   # Houston vs Golden State
    ]
    east_matchups = [
        (east_teams[0], east_teams[7]),  # Cleveland vs Atlanta
        (east_teams[3], east_teams[4]),  # Indiana vs Milwaukee
        (east_teams[2], east_teams[5]),  # Knicks vs Detroit
        (east_teams[1], east_teams[6])   # Boston vs Orlando
    ]
    
    # Set opponent probabilities to sum to 100% per series
    for t1, t2 in west_matchups + east_matchups:
        if t1 in first_round_probs and t2 in first_round_probs:
            prob_t1 = first_round_probs[t1]
            first_round_probs[t2] = 100 - prob_t1
        elif t1 in first_round_probs:
            first_round_probs[t2] = 100 - first_round_probs[t1]
        elif t2 in first_round_probs:
            first_round_probs[t1] = 100 - first_round_probs[t2]
    
    results['first_round'] = first_round_probs
    
    print(f"Successful simulations: {successful_sims}/{n_simulations}")
    
    return results

# Define playoff teams based on provided seeds
west_teams = [
    'Oklahoma City Thunder',   # 1
    'Houston Rockets',        # 2
    'Los Angeles Lakers',     # 3
    'Denver Nuggets',         # 4
    'LA Clippers',            # 5
    'Minnesota Timberwolves', # 6
    'Golden State Warriors',  # 7
    'Memphis Grizzlies'       # 8
]

east_teams = [
    'Cleveland Cavaliers',    # 1
    'Boston Celtics',         # 2
    'New York Knicks',        # 3
    'Indiana Pacers',         # 4
    'Milwaukee Bucks',        # 5
    'Detroit Pistons',        # 6
    'Orlando Magic',          # 7
    'Miami Heat'              # 8
]

# Verify all teams exist in playoff stats
all_teams = west_teams + east_teams
missing_teams = [team for team in all_teams if team not in playoff_stats_2025['Team'].values]
if missing_teams:
    print(f"Error: The following teams are missing from nba_advanced_stats_2025.csv: {missing_teams}")
    exit(1)

print("Western Conference Seeds:", west_teams)
print("Eastern Conference Seeds:", east_teams)

# Run simulation
n_simulations = 1000
results = run_monte_carlo(east_teams, west_teams, n_simulations)

if results is None:
    exit(1)

# Print results
print("\nMonte Carlo Simulation Results (Probabilities, Fixed Bracket, Series Win Probabilities):")
for stage in results:
    print(f"\n{stage.replace('_', ' ').title()}:")
    stage_sum = 0
    for team, prob in sorted(results[stage].items(), key=lambda x: x[1], reverse=True):
        print(f"  {team}: {prob:.2f}%")
        stage_sum += prob
    #print(f"  Sum: {stage_sum:.2f}%")

# Save results to CSV
for stage in results:
    pd.DataFrame.from_dict(results[stage], orient='index', columns=['Probability']).to_csv(f'{stage}_probabilities_2025_fixed_bracket_series.csv')

Loading model...
Model loaded and compiled
Loading scaler...
C:\Users\jonla\NBA_Playoffs_Series_Predictor\Models\Scaler loaded
Loading playoff stats...
Normalizing PIE (dividing by 100)
Normalizing eFG% (dividing by 100)
Normalizing OREB% (dividing by 100)
Normalizing DREB% (dividing by 100)
Normalizing TS% (dividing by 100)
Playoff stats loaded
Western Conference Seeds: ['Oklahoma City Thunder', 'Houston Rockets', 'Los Angeles Lakers', 'Denver Nuggets', 'LA Clippers', 'Minnesota Timberwolves', 'Golden State Warriors', 'Memphis Grizzlies']
Eastern Conference Seeds: ['Cleveland Cavaliers', 'Boston Celtics', 'New York Knicks', 'Indiana Pacers', 'Milwaukee Bucks', 'Detroit Pistons', 'Orlando Magic', 'Miami Heat']
Successful simulations: 1000/1000

Monte Carlo Simulation Results (Probabilities, Fixed Bracket, Series Win Probabilities):

First Round:
  Cleveland Cavaliers: 70.50%
  Oklahoma City Thunder: 67.70%
  Boston Celtics: 62.60%
  Houston Rockets: 54.60%
  LA Clippers: 54.40%
  Los A